# Step 0. 환경 검증
Terraform으로 생성한 AWS 리소스가 정상적으로 동작하는지 확인합니다.

## 검증 항목
1. AWS 자격 증명 (STS 세션 토큰)
2. OpenSearch Serverless 연결
3. Bedrock 임베딩 모델 호출
4. Bedrock LLM 모델 호출
5. OpenSearch Dashboard 접속 URL

In [ ]:
!pip install -q boto3==1.38.46 opensearch-py==2.8.0

## 설정

In [26]:
import os

HOST = os.environ.get("OPENSEARCH_HOST", "")
if not HOST:
    HOST = input("OPENSEARCH_HOST 값을 입력하세요: ").strip()
if not HOST:
    raise ValueError("❌ OPENSEARCH_HOST 값이 필요합니다.")

DEFAULT_REGION = "ap-northeast-2"
BEDROCK_REGION = "us-east-1"
PROFILE = os.environ.get("AWS_PROFILE", "skku-opensearch-session")

## 1. AWS 자격 증명 확인
`get-session-token.sh`로 발급한 STS 임시 토큰이 유효한지 확인합니다.

In [21]:
import boto3

try:
    session = boto3.Session(profile_name=PROFILE)
    sts = session.client("sts")
    identity = sts.get_caller_identity()
    print("✅ AWS 자격 증명 확인 성공")
    print(f"   Account: {identity['Account']}")
    print(f"   Arn:     {identity['Arn']}")
except Exception as e:
    print(f"❌ AWS 자격 증명 확인 실패: {e}")
    print("   → ./get-session-token.sh 를 먼저 실행해주세요.")

✅ AWS 자격 증명 확인 성공
   Account: 666803869796
   Arn:     arn:aws:iam::666803869796:user/kdt-admin


## 2. OpenSearch Serverless 연결 확인
Terraform으로 생성한 OpenSearch Serverless 컬렉션에 연결하고 인덱스 목록을 조회합니다.

In [22]:
from opensearchpy import OpenSearch, AWSV4SignerAuth, RequestsHttpConnection

try:
    credentials = session.get_credentials()
    auth = AWSV4SignerAuth(credentials, DEFAULT_REGION, "aoss")

    client = OpenSearch(
        hosts=[{"host": HOST, "port": 443}],
        http_auth=auth,
        use_ssl=True,
        verify_certs=True,
        connection_class=RequestsHttpConnection,
        timeout=300,
    )

    # 테스트 인덱스 존재 여부 확인 (AOSS는 cat API를 지원하지 않음)
    test_index = "connection-test"
    exists = client.indices.exists(index=test_index)
    print("✅ OpenSearch 연결 성공")
    print(f"   Host: {HOST}")

    # 실습에서 사용하는 인덱스 확인
    for idx_name in ["json-test", "vector-test", "bedrock-test", "bedrock-image-test"]:
        idx_exists = client.indices.exists(index=idx_name)
        status = "있음" if idx_exists else "없음"
        print(f"   - {idx_name}: {status}")

except Exception as e:
    print(f"❌ OpenSearch 연결 실패: {e}")
    print("   → HOST 값이 올바른지, Terraform이 정상 완료되었는지 확인해주세요.")
    print("   → ./get-session-token.sh 를 다시 실행해보세요.")

✅ OpenSearch 연결 성공
   Host: 1exd0ey01a7aziy1y4k8.ap-northeast-2.aoss.amazonaws.com
   - json-test: 없음
   - vector-test: 없음
   - bedrock-test: 없음
   - bedrock-image-test: 없음


## 3. Bedrock 임베딩 모델 호출 확인
Amazon Bedrock Titan 임베딩 모델이 정상적으로 응답하는지 확인합니다.

In [23]:
import json

try:
    bedrock = session.client(
        service_name="bedrock-runtime",
        region_name=BEDROCK_REGION,
    )

    body = json.dumps({"inputText": "테스트 문장입니다."})
    response = bedrock.invoke_model(
        body=body,
        modelId="amazon.titan-embed-text-v2:0",
        accept="application/json",
        contentType="application/json",
    )
    result = json.loads(response["body"].read())
    embedding = result["embedding"]

    print("✅ Bedrock 임베딩 모델 호출 성공")
    print(f"   모델: amazon.titan-embed-text-v2:0")
    print(f"   벡터 차원: {len(embedding)}")
    print(f"   벡터 샘플: {embedding[:5]}...")
except Exception as e:
    print(f"❌ Bedrock 호출 실패: {e}")
    print(f"   → Bedrock 리전({BEDROCK_REGION})에 모델 접근 권한이 있는지 확인해주세요.")

✅ Bedrock 임베딩 모델 호출 성공
   모델: amazon.titan-embed-text-v2:0
   벡터 차원: 1024
   벡터 샘플: [-0.041096631437540054, 0.014900458976626396, 0.07102346420288086, 0.03049541637301445, 0.03322906792163849]...


## 4. Bedrock LLM 모델 호출 확인
실습에서 사용할 Claude Sonnet 4.5 모델이 정상적으로 응답하는지 확인합니다.

In [ ]:
LLM_MODEL_ID = "us.anthropic.claude-sonnet-4-5-20250929-v1:0"

try:
    body = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 100,
        "messages": [
            {"role": "user", "content": [{"type": "text", "text": "안녕? 한 줄로 대답해줘."}]}
        ],
    })

    response = bedrock.invoke_model(
        body=body,
        modelId=LLM_MODEL_ID,
        accept="application/json",
        contentType="application/json",
    )
    result = json.loads(response["body"].read())
    answer = result["content"][0]["text"]

    print("✅ Bedrock LLM 모델 호출 성공")
    print(f"   모델: {LLM_MODEL_ID}")
    print(f"   응답: {answer}")
except Exception as e:
    print(f"❌ Bedrock LLM 호출 실패: {e}")
    print(f"   → Bedrock 리전({BEDROCK_REGION})에서 Claude 모델 접근 권한이 있는지 확인해주세요.")

## 5. OpenSearch Dashboard 접속
아래 URL을 브라우저에서 열어 OpenSearch Dashboard에 접속할 수 있는지 확인하세요.

In [24]:
dashboard_url = f"https://{HOST}/_dashboards"
print(f"🔗 OpenSearch Dashboard URL:")
print(f"   {dashboard_url}")
print()
print("   브라우저에서 위 URL을 열어 Dashboard에 접속해보세요.")
print("   (AWS 콘솔 로그인 상태여야 접속 가능합니다)")

🔗 OpenSearch Dashboard URL:
   https://1exd0ey01a7aziy1y4k8.ap-northeast-2.aoss.amazonaws.com/_dashboards

   브라우저에서 위 URL을 열어 Dashboard에 접속해보세요.
   (AWS 콘솔 로그인 상태여야 접속 가능합니다)


## 6. 설정 저장
위에서 검증한 설정값을 `config.json`에 저장합니다. Step1~5에서 이 파일을 읽어 사용합니다.

In [27]:
config = {
    "OPENSEARCH_HOST": HOST,
    "DEFAULT_REGION": DEFAULT_REGION,
    "BEDROCK_REGION": BEDROCK_REGION,
    "PROFILE": PROFILE,
}

config_path = "../config.json"
with open(config_path, "w") as f:
    json.dump(config, f, indent=2)

print(f"✅ 설정이 {config_path} 에 저장되었습니다.")
print(json.dumps(config, indent=2))

✅ 설정이 ../config.json 에 저장되었습니다.
{
  "OPENSEARCH_HOST": "1exd0ey01a7aziy1y4k8.ap-northeast-2.aoss.amazonaws.com",
  "DEFAULT_REGION": "ap-northeast-2",
  "BEDROCK_REGION": "us-east-1",
  "PROFILE": "skku-opensearch-session"
}


## 검증 결과 요약
위 5개 항목을 모두 실행한 후, 모든 항목에 ✅가 표시되면 실습 준비가 완료된 것입니다.

| 항목 | 실패 시 확인사항 |
|------|------------------|
| AWS 자격 증명 | `./get-session-token.sh` 실행 |
| OpenSearch 연결 | Terraform 정상 완료 여부, HOST 값 확인 |
| Bedrock 임베딩 | Bedrock 모델 접근 권한 (AWS 콘솔에서 모델 활성화) |
| Bedrock LLM | Bedrock 리전(us-east-1)에서 Claude 모델 활성화 여부 |
| Dashboard 접속 | AWS 콘솔 로그인 상태 확인 |